In [1]:
from east_dataset import EASTDataset
from dataset import SceneTextDataset

trainset = SceneTextDataset(
        '/opt/ml/input/data/medical',
        split='train',
        image_size=2048,
        crop_size=1024,
        ignore_tags=['masked', 'excluded-region', 'maintable', 'stamp'],
        normalize=False,
        color_jitter=False
    )
trainset = EASTDataset(trainset, to_tensor=False)

In [2]:
from tqdm import tqdm  
import pickle

for i in tqdm(range(len(trainset))):
    g = trainset.__getitem__(i)
    with open(file=f'/opt/ml/input/data/pickle_data/train/{i}.pkl',mode='wb') as f:
        pickle.dump(g,f, protocol=pickle.HIGHEST_PROTOCOL)

  3%|▎         | 16/500 [03:49<1:55:57, 14.37s/it]


KeyboardInterrupt: 

In [ ]:
for i in tqdm(range(len(valset))):
    g = valset.__getitem__(i)
    with open(file=f'/opt/ml/input/data/pickle_data/val/{i}.pkl',mode='wb') as f:
        pickle.dump(g,f)

In [ ]:
from torch.utils.data import Dataset
import os 
import torch 

class CustomDataset(Dataset):
    def __init__(self, data_dir, split):
        self.datadir = data_dir
        self.split = split 
        datalist = os.listdir(data_dir)
        self.datalist = [d for d in datalist if d.endswith(".pkl")]
        self.datalist.sort()

    def __getitem__(self, idx):
        with open(file=f'/opt/ml/input/data/pickle_data/{self.split}/{idx}.pkl',mode='rb') as f:
            data = pickle.load(f)
        image, score_map, geo_map, roi_mask = data
        transform = A.Compose([A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

        image = transform(image=image)['image']
        image = torch.Tensor(image).permute(2, 0, 1)

        score_map = torch.Tensor(score_map).permute(2, 0, 1)
        geo_map = torch.Tensor(geo_map).permute(2, 0, 1)
        roi_mask = torch.Tensor(roi_mask).permute(2, 0, 1)

        return image, score_map, geo_map, roi_mask
    
    def __len__(self):
        return len(self.datalist)

## Val Datset plk
deteval 을 계산하는 val dataset 에서 사용

In [ ]:
from dataset import ValDataset

val_dataset = ValDataset(root_dir="/opt/ml/input/data/medical")

input_img, word_bboxes, roi_mask, gt_score_map, gt_geo_map, ori_size, transcriptions = next(iter(val_dataset))

In [ ]:
print(f"input_img      {str(type(input_img)):<25s} {str(input_img.shape):<25s}")
print(f"word_bboxes    {str(type(word_bboxes)):<25s} {str(word_bboxes.shape):<25s}")
print(f"roi_mask       {str(type(roi_mask)):<25s} {str(roi_mask.shape):<25s}")
print(f"gt_score_map   {str(type(gt_score_map)):<25s} {str(gt_score_map.shape):<25s}")
print(f"gt_geo_map     {str(type(gt_geo_map)):<25s} {str(gt_geo_map.shape):<25s}")
print(f"ori_size       {str(type(ori_size)):<25s} {str(ori_size):<25s}")
print(f"transcriptions {str(type(transcriptions)):<25s} {str(len(transcriptions)):<25s}")

In [ ]:
from tqdm import tqdm  
import pickle

for i in tqdm(range(len(val_dataset))):
    g = val_dataset.__getitem__(i)
    with open(file=f'/opt/ml/input/data/medical/pickle_data/val/{i}.pkl',mode='wb') as f:
        pickle.dump(g,f, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
from torch.utils.data import Dataset
import os 

class CustomDataset(Dataset):
    def __init__(self, data_dir = "/opt/ml/input/data/medical/pickle_data/", split='val'):
        self.data_dir = data_dir
        self.split = split 
        datalist = os.listdir(os.path.join(self.data_dir,self.split))
        self.data_list = [d for d in datalist if d.endswith(".pkl")]
        self.data_list.sort()

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir,self.split,self.data_list[idx])
        print(file_path)
        with open(file=file_path,mode='rb') as f:
            data = pickle.load(f)
        input_img, word_bboxes, roi_mask, gt_score_map, gt_geo_map, ori_size, transcriptions = data

        return input_img, word_bboxes, roi_mask, gt_score_map, gt_geo_map, ori_size, transcriptions
    
    def __len__(self):
        return len(self.datalist)

In [ ]:
val_dataset = CustomDataset()

input_img, word_bboxes, roi_mask, gt_score_map, gt_geo_map, ori_size, transcriptions = next(iter(val_dataset))

In [ ]:
print(f"input_img      {str(type(input_img)):<25s} {str(input_img.shape):<25s}")
print(f"word_bboxes    {str(type(word_bboxes)):<25s} {str(word_bboxes.shape):<25s}")
print(f"roi_mask       {str(type(roi_mask)):<25s} {str(roi_mask.shape):<25s}")
print(f"gt_score_map   {str(type(gt_score_map)):<25s} {str(gt_score_map.shape):<25s}")
print(f"gt_geo_map     {str(type(gt_geo_map)):<25s} {str(gt_geo_map.shape):<25s}")
print(f"ori_size       {str(type(ori_size)):<25s} {str(ori_size):<25s}")
print(f"transcriptions {str(type(transcriptions)):<25s} {str(len(transcriptions)):<25s}")